# STARDUST (no cell segmentation)

This Jupyter Notebook provides an interactive version of STARDUST for analysis without cell segmentation (ROA-based or whole cell-based). Functions are defined and imported from util.py. Use help() to see the documentation. For a more detailed description of the STARDUST pipeline, please see the biorxiv paper and github page for more information. 

## 1. Environment set up

In [ ]:
import pandas as pd, numpy as np, seaborn as sns
from src.STARDUST.util import * 

## 2. Data input

Run the next code block to read in input files and information of the experiment. Enter the information accordingly in the prompted text boxes. 

In [ ]:
time_series_path, _, _, output_path = prompt_input(analysis_type = 'cell')
drug_frame, frame_rate, spatial_resolution = get_metadata()

## 3. Signal preprocessing

In [ ]:
# find raw traces and create filtered traces
raw_traces, filtered_traces = raw_to_filtered(time_series_path)

In [ ]:
ROA_count, frame_count = check_traces(filtered_traces)

## 4. Baseline determination and signal detection

In [ ]:
# baseline determination
dff_traces, baselines, thresholds, signal_frames, signal_boundaries, signal_threshold = iterative_baseline(filtered_traces, include_incomplete = True, baseline_end = 200)

## Checkpoint 1: dF/F traces heatmap

Visualze dF/F traces using heatmap. Each row represents one cell and each column represents one frame.  

Note that for heatmap coloring, *vmin* is set at 0, and *vmax* is set at (signal_threshold + 2) * average thresholds across all cells. For example, if the signal_threshold is set at 3SD, any transients that has a dF/F value above baseline + 5SD will be colored red to facilitate visualization. You can adjust the vmax parameter if needed. 

In [ ]:
sns.heatmap(dff_traces, vmin = 0, vmax = (signal_threshold + 2) * thresholds.mean(), 
            xticklabels=100, yticklabels= False, cmap = 'jet');

## 5. Signal feature extraction

In [ ]:
signal_features = analyze_signal(dff_traces, signal_frames, signal_boundaries, frame_rate, drug_frame)

In [ ]:
signal_features.head()

## Checkpoint 2: Individual traces
Use the inspect_trace() function to visualize all traces. 

In [ ]:
all_cells = range(1, ROA_count + 1)
inspect_trace(all_cells, dff_traces, baselines, thresholds, drug_frame)

## 6. Data output

In [ ]:
metadata = metadata_todf()
output_data(output_path, metadata, dff_traces, signal_features, save_as = 'csv')